In [ ]:
import signal
import threading
from collections import defaultdict
from typing import Dict

from ipyleaflet import Circle, LayerGroup, Map
from palettable.colorbrewer.qualitative import Set1_9, Set3_6
from visionapi.messages_pb2 import SaeMessage
from visionlib.pipeline.consumer import RedisConsumer

# Setup graceful exit on SIGINT
stop_event = threading.Event()

def sig_handler(signum, _):
    signame = signal.Signals(signum).name
    print(f'Caught signal {signame} ({signum}). Exiting...')
    stop_event.set()

signal.signal(signal.SIGTERM, sig_handler)
signal.signal(signal.SIGINT, sig_handler)


CLS_CMAP = Set1_9.hex_colors
STREAM_CMAP = Set3_6.hex_colors

# Map setup
INIT_CENTER = (39.96871094145749, -86.12691605973491)
DETECTED_CENTER = None

m = Map(center=INIT_CENTER, zoom=14)
marker_layer = LayerGroup()
m.add(marker_layer)
m.layout.height = '800px'
display(m)

displayed_objects: Dict[str, Dict[str, Circle]] = defaultdict(lambda: defaultdict(dict))

# Start listening to SAE messages
consume = RedisConsumer('localhost', 6379, ['geomapper:Monon1stStreetSB', 'geomapper:Monon1stStreetNB'], block=200)

with consume:
    for stream_key, proto_data in consume():

        if stop_event.is_set():
            break

        if stream_key is None:
            continue

        sae_msg = SaeMessage()
        sae_msg.ParseFromString(proto_data)

        # Update map center with the first detected object (should only happen once)
        if DETECTED_CENTER is None and len(sae_msg.detections) > 0:
            DETECTED_CENTER = (sae_msg.detections[0].geo_coordinate.latitude, sae_msg.detections[0].geo_coordinate.longitude)
            m.center = DETECTED_CENTER
            m.zoom = 19

        # Remove objects from cache and map that were rendered before but do not exist anymore
        removed_objects = list(filter(lambda id: id not in map(lambda d: d.object_id, sae_msg.detections), displayed_objects[stream_key].keys()))
        for id in removed_objects:
            removed = displayed_objects[stream_key].pop(id, None)
            if removed is not None:
                marker_layer.remove(removed)

        # Add new objects / Update existing objects locations
        for detection in sae_msg.detections:
            lat, lon = detection.geo_coordinate.latitude, detection.geo_coordinate.longitude
            if detection.object_id in displayed_objects[stream_key]:
                displayed_objects[stream_key][detection.object_id].location = (lat, lon)
            else:
                marker = Circle(location=[lat, lon], radius=2, color=STREAM_CMAP[hash(stream_key) % len(STREAM_CMAP)], fill_color=CLS_CMAP[detection.class_id], fill_opacity=0.8)
                displayed_objects[stream_key][detection.object_id] = marker
                marker_layer.add(marker)